## Script to calculate the average precipitation rate for each model grid cell.

In [1]:
import numpy as np
import pandas as pd
from IPython import parallel as pr

In [2]:
c = pr.Client() # start the parallel environment

In [3]:
c.ids # check the number of engines available

[0, 1, 2, 3]

In [4]:
view = c.load_balanced_view() # access the first 4 engines

In [10]:
%%px
import pandas as pd
import numpy as np
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
forcing_columns = ['P','Tmax','Tmin','W']
# bring in the data frame of file paths
files = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/forcing_fluxes_filenames_lat_lon_index.df') 

In [11]:
dates = pd.read_pickle('/Volumes/data/Theo/projects/Budyko_vic/timecode.pcl')
forcing_columns = ['P','Tmax','Tmin','W']

In [12]:
files = pd.read_pickle('./forcing_fluxes_filenames_lat_lon_index.df') # bring in the data frame of file paths

In [13]:
# define a long term P function
def p_rate(idx,fr):
    
    force = pd.read_table(fr,sep=' ', names = forcing_columns) # read forcings
    force.index = pd.DatetimeIndex(dates)
    
    Prate = force.loc[force.P>0,'P'].mean()
    
    return idx,Prate

In [26]:
#res = view.map(p_rate,files.indexer[0:1000],files.forcing[0:1000]) # map the function to all the basins over the engines
res = view.map(p_rate,files.indexer,files.forcing)

In [28]:
res.ready()

True

In [29]:
res.progress

309673

In [30]:
res.elapsed/60.

40.5027762

In [31]:
idx,prate = zip(*res.result)

In [32]:
np.savez_compressed('/Volumes/data/Theo/projects/Budyko_vic/data/precip_rate.npz',idx=idx,prate=prate)